In [ ]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=60bacdbccafa3c5c1c0038d81e85d03523e14dab9dec65bca9a4d73816b3a066
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=HcRsVrG6uaPZqq4G7u133M3L4eP8JG&prompt=consent&token_usage=remote&access_type=offline&code_challenge=1bAELYCKpJ_VxYdQunbvUPBAwoZ80BgnRSsyluBEPtA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrDPDUQhUy3UjhYQEVTdO-aStMxFAElBqwYUwlJt-hQUw0enBEScmSCnuS1ERkmgyg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
df = pd.read_excel('PEP_ingresso.xlsx')
df

,Ano,Ident. GDF,Tipo de Ingresso',Quantidade de Ingressos,Regime Jurídico,Sexo,Grupo do Cargo,Natureza Jurídica
0,Total de Seleção,Total de Seleção,Total de Seleção,950846.0,Total de Seleção,Total de Seleção,Total de Seleção,Total de Seleção
1,2024,Gov. Federal,Processo Seletivo,128.0,Contrato Temporário,F,Aposentado Contrato Temporario Inss,ADMINISTRACAO DIRETA FEDERAL
2,2024,Gov. Federal,Processo Seletivo,941.0,Contrato Temporário,F,Carreira De Magisterio Superior,AUTARQUIA FEDERAL
3,2024,Gov. Federal,Processo Seletivo,408.0,Contrato Temporário,F,Carreira De Magisterio Superior,FUNDACAO FEDERAL
4,2024,Gov. Federal,Processo Seletivo,868.0,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,AUTARQUIA FEDERAL
...,...,...,...,...,...,...,...,...
5384,1995,GDF,Concurso,973.0,Regime Militar,M,Espec.Em Polit.Publ.E Gest.Governamental,SERVICO PUBLICO ESTADUAL
5385,1995,Gov. Federal,Transposição,2.0,Regime Militar,M,Militar (Ex-Territorios) Vpext,ADMINISTRACAO DIRETA FEDERAL
5386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5387,Selection Status:,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
##Remover colunas com dados despadronizados devido ao sheets
df = df.drop([0, 5386, 5387, 5388])
df

,Ano,Ident. GDF,Tipo de Ingresso',Quantidade de Ingressos,Regime Jurídico,Sexo,Grupo do Cargo,Natureza Jurídica
1,2024,Gov. Federal,Processo Seletivo,128.0,Contrato Temporário,F,Aposentado Contrato Temporario Inss,ADMINISTRACAO DIRETA FEDERAL
2,2024,Gov. Federal,Processo Seletivo,941.0,Contrato Temporário,F,Carreira De Magisterio Superior,AUTARQUIA FEDERAL
3,2024,Gov. Federal,Processo Seletivo,408.0,Contrato Temporário,F,Carreira De Magisterio Superior,FUNDACAO FEDERAL
4,2024,Gov. Federal,Processo Seletivo,868.0,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,AUTARQUIA FEDERAL
5,2024,Gov. Federal,Processo Seletivo,19.0,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,ADMINISTRACAO DIRETA FEDERAL
...,...,...,...,...,...,...,...,...
5381,1995,GDF,Concurso,5.0,Estatutário,M,-,-
5382,1995,Gov. Federal,Concurso,1.0,Estatutário,M,-,AUTARQUIA ESPECIAL
5383,1995,GDF,Concurso,132.0,Regime Militar,F,Espec.Em Polit.Publ.E Gest.Governamental,SERVICO PUBLICO ESTADUAL
5384,1995,GDF,Concurso,973.0,Regime Militar,M,Espec.Em Polit.Publ.E Gest.Governamental,SERVICO PUBLICO ESTADUAL


In [ ]:
df['Ano'] = df['Ano'].astype(int)
df['Quantidade de Ingressos'] = df['Quantidade de Ingressos'].astype(int)
df = df[(df['Ano']>2007) & (df['Ano'] <= 2024)]
df

,Ano,Ident. GDF,Tipo de Ingresso',Quantidade de Ingressos,Regime Jurídico,Sexo,Grupo do Cargo,Natureza Jurídica
1,2024,Gov. Federal,Processo Seletivo,128,Contrato Temporário,F,Aposentado Contrato Temporario Inss,ADMINISTRACAO DIRETA FEDERAL
2,2024,Gov. Federal,Processo Seletivo,941,Contrato Temporário,F,Carreira De Magisterio Superior,AUTARQUIA FEDERAL
3,2024,Gov. Federal,Processo Seletivo,408,Contrato Temporário,F,Carreira De Magisterio Superior,FUNDACAO FEDERAL
4,2024,Gov. Federal,Processo Seletivo,868,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,AUTARQUIA FEDERAL
5,2024,Gov. Federal,Processo Seletivo,19,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,ADMINISTRACAO DIRETA FEDERAL
...,...,...,...,...,...,...,...,...
2792,2008,GDF,Concurso,9,Estatutário,M,Policia Civil - Gdf,SERVICO PUBLICO ESTADUAL
2793,2008,Gov. Federal,Concurso,2,Estatutário,M,Receita Federal Brasil,ADMINISTRACAO DIRETA FEDERAL
2794,2008,Gov. Federal,Concurso,2,Estatutário,M,Receita Federal Do Brasil,ADMINISTRACAO DIRETA FEDERAL
2795,2008,GDF,Concurso,10,Regime Militar,F,Espec.Em Polit.Publ.E Gest.Governamental,SERVICO PUBLICO ESTADUAL


In [ ]:
df=df[df['Ident. GDF']=='Gov. Federal']
df

,Ano,Ident. GDF,Tipo de Ingresso',Quantidade de Ingressos,Regime Jurídico,Sexo,Grupo do Cargo,Natureza Jurídica
1,2024,Gov. Federal,Processo Seletivo,128,Contrato Temporário,F,Aposentado Contrato Temporario Inss,ADMINISTRACAO DIRETA FEDERAL
2,2024,Gov. Federal,Processo Seletivo,941,Contrato Temporário,F,Carreira De Magisterio Superior,AUTARQUIA FEDERAL
3,2024,Gov. Federal,Processo Seletivo,408,Contrato Temporário,F,Carreira De Magisterio Superior,FUNDACAO FEDERAL
4,2024,Gov. Federal,Processo Seletivo,868,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,AUTARQUIA FEDERAL
5,2024,Gov. Federal,Processo Seletivo,19,Contrato Temporário,F,Carreira Magist Ens Bas Tec Tecnol,ADMINISTRACAO DIRETA FEDERAL
...,...,...,...,...,...,...,...,...
2789,2008,Gov. Federal,Concurso,1,Estatutário,M,Plano Especial De Cargos Da Cultura,ADMINISTRACAO DIRETA FEDERAL
2790,2008,Gov. Federal,Concurso,102,Estatutário,M,Plano Geral De Cargos Do Poder Executivo,ADMINISTRACAO DIRETA FEDERAL
2791,2008,Gov. Federal,Concurso,99,Estatutário,M,Plano Geral De Cargos Poder Executivo-Ni,ADMINISTRACAO DIRETA FEDERAL
2793,2008,Gov. Federal,Concurso,2,Estatutário,M,Receita Federal Brasil,ADMINISTRACAO DIRETA FEDERAL


In [ ]:
df=df[['Ano',  "Tipo de Ingresso'", 'Regime Jurídico', 'Sexo',
       'Grupo do Cargo', 'Quantidade de Ingressos']]
df['Sexo']= np.where(df['Sexo']=='F',"Feminino",df['Sexo'])

<ipython-input-26-633664c9b32f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sexo']= np.where(df['Sexo']=='F',"Feminino",df['Sexo'])


In [ ]:
df['Sexo']= np.where(df['Sexo']=='M',"Masculino",df['Sexo'])
df

<ipython-input-27-a5d78bd19c56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sexo']= np.where(df['Sexo']=='M',"Masculino",df['Sexo'])


,Ano,Tipo de Ingresso',Regime Jurídico,Sexo,Grupo do Cargo,Quantidade de Ingressos
1,2024,Processo Seletivo,Contrato Temporário,Feminino,Aposentado Contrato Temporario Inss,128
2,2024,Processo Seletivo,Contrato Temporário,Feminino,Carreira De Magisterio Superior,941
3,2024,Processo Seletivo,Contrato Temporário,Feminino,Carreira De Magisterio Superior,408
4,2024,Processo Seletivo,Contrato Temporário,Feminino,Carreira Magist Ens Bas Tec Tecnol,868
5,2024,Processo Seletivo,Contrato Temporário,Feminino,Carreira Magist Ens Bas Tec Tecnol,19
...,...,...,...,...,...,...
2789,2008,Concurso,Estatutário,Masculino,Plano Especial De Cargos Da Cultura,1
2790,2008,Concurso,Estatutário,Masculino,Plano Geral De Cargos Do Poder Executivo,102
2791,2008,Concurso,Estatutário,Masculino,Plano Geral De Cargos Poder Executivo-Ni,99
2793,2008,Concurso,Estatutário,Masculino,Receita Federal Brasil,2


In [ ]:
df = df.rename(columns={'Ano':'ano',"Tipo de Ingresso'":'tipo_ingresso','Regime Jurídico':'regime_juridico','Sexo':'genero','Grupo do Cargo':'grupo_cargo','Quantidade de Ingressos':'quantidade_ingressos'})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2660 entries, 1 to 2794
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ano                   2660 non-null   int64 
 1   tipo_ingresso         2660 non-null   object
 2   regime_juridico       2660 non-null   object
 3   genero                2660 non-null   object
 4   grupo_cargo           2660 non-null   object
 5   quantidade_ingressos  2660 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 145.5+ KB


In [ ]:
df['regime_juridico'].unique()

array(['Contrato Temporário', 'Estatutário', 'Médico Residente',
       'Programa Mais Médicos', 'Residência Multi Profissional',
       'Regime Militar'], dtype=object)

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('tipo_ingresso','STRING',description='forma de ingresso no serviço público'),
 bigquery.SchemaField('regime_juridico','STRING',description='regime jurídico'),
 bigquery.SchemaField('genero','STRING',description='Qual gênero do ingressante'),
 bigquery.SchemaField('grupo_cargo','STRING',description='a qual grupo de cargo pertence'),
 bigquery.SchemaField('quantidade_ingressos','INTEGER',description='Quantidade de ingressos')
 ]

In [ ]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('atracao_pre_selecao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
table_ref = dataset_ref.table('PEP_GF_ingressos_v3') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=4b3ceda6-88b5-47cb-95a2-a81369db1a71>

In [ ]:
df['genero'].unique()

array(['Feminino', 'Masculino'], dtype=object)